# importing libraries 

In [15]:
import numpy as np
import pandas as pd

## how to read and work with an excel_file for example

In [ ]:
a='U'
b='98'
tr98=[]
for file in pd.ExcelFile('data_sets/project1/'+a+b+'.xlsx').sheet_names:
   tr98.append(pd.read_excel('data_sets/project1/'+a+b+'.xlsx',sheet_name=file))

## Connecting sheets to each other with merge

In [ ]:
dftr98=tr98[0]
#dftr98=dftr98.merge(tr98[1],on='Address',how='left')
dftr98=dftr98.merge(tr98[2],on='Address',how='left')
# here i change the values to cost1, cost2, ...
for i in range(3,17):
    if i != 12:
        tr98[i]['value'] = tr98[i]['value'].apply(object_to_int)
        dftr98=dftr98.merge(tr98[i].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Cost'+str(i-2)}),on='Address',how='left')
    
#house  price 1
dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==41)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'HousePrice1'}),on='Address',how='left')
#house price 2
dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==42)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'HousePrice2'}),on='Address',how='left')
# fuel
dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==45)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Fuel'}),on='Address',how='left')
#car fuel
dftr98=dftr98.merge(tr98[9][tr98[9]['code'].apply(lambda x:x//100==722)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Car_Fuel'}),on='Address',how='left')
#benzin
dftr98=dftr98.merge(tr98[9][tr98[9]['code'].apply(lambda x:x==72211)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Benzin'}),on='Address',how='left')


## calculating the costs_sum column

In [ ]:
dftr98['Costs_sum'] = 0
cost_str = 'Cost'
for i in range(1,15):
    if i != 10:
        dftr98['Costs_sum'] += dftr98[cost_str + str(i)]

## There are 4 type of income but all of the families doesnt have all Four income type 
### then i consider the fasl1 as the "main fasl" and fill the empty fasl1(s) with other fasls

In [ ]:
def custom_fasl(row):
    fasl_str = 'Fasl'
    for i in range(1,5):
        if ~np.isnan(row[fasl_str + str(i)]):
            row['Fasl1'] = row[fasl_str + str(i)]
            break
    return row

In [ ]:

temp = tr98[18].groupby('Address').agg({'income_s_y':'sum'}).rename(columns={'income_s_y':'Income_s_y'})
tr98[18] = tr98[18].rename({'Fasl':'Fasl2'}, axis=1)
temp_2 = tr98[18].groupby('Address')[['Fasl2']].first()
temp = temp.merge(temp_2, on='Address', how='left')
dftr98 = dftr98.merge(temp, on='Address', how='left')

tr98[19]['income_pension'] = tr98[19]['income_pension'].apply(object_to_int)
tr98[19]['income_rent'] = tr98[19]['income_rent'].apply(object_to_int)
temp = tr98[19].groupby('Address').agg({'income_transfer':'sum', 'income_pension':'sum',
                                        'income_rent':'sum','income_aid':'sum',
                                        'income_interest':'sum', 'income_resale':'sum'})
temp['sum03_incomes'] = temp['income_transfer']+temp['income_pension']+temp['income_rent']+temp['income_aid']+temp['income_interest']+temp['income_resale']
temp.drop(['income_transfer', 'income_pension','income_rent','income_aid','income_interest', 'income_resale'], axis=1, inplace=True)
tr98[19] = tr98[19].rename({'Fasl':'Fasl3'}, axis=1)
temp_2 = tr98[19].groupby('Address')[['Fasl3']].first()
temp = temp.merge(temp_2, on='Address', how='left')
dftr98 = dftr98.merge(temp, on='Address', how='left')

temp = tr98[20].groupby('Address').agg({'subsidy':'sum'}).rename(columns={'subsidy':'Subsidy'})
tr98[20] = tr98[20].rename({'Fasl':'Fasl4'}, axis=1)
temp_2 = tr98[20].groupby('Address')[['Fasl4']].first()
temp = temp.merge(temp_2, on='Address', how='left')
dftr98 = dftr98.merge(temp, on='Address', how='left')


## dropping the columns that has alot of Nan values

In [ ]:
column_to_drop = ['TakmilDescA','TakmilDescB', 'TakmilDescC','JaygozinDescA','JaygozinDescB','JaygozinDescC','BlkAbdJaygozin','RadifJaygozin','Jaygozin']
existing_column = [col for col in column_to_drop if col in dftr98.columns]
dftr98 = dftr98.drop(existing_column, axis = 1)

## filling Nan values

In [ ]:
dftr98[['HousePrice1', 'HousePrice2', 'Fuel', 'Car_Fuel', 'Benzin']] = dftr98[['HousePrice1', 'HousePrice2', 'Fuel', 'Car_Fuel', 'Benzin']].fillna(0)
dftr98['sum_netincome_w_m'] = dftr98['sum_netincome_w_m'].fillna(0)
dftr98['sum_netincome_w_y'] = dftr98['sum_netincome_w_y'].fillna(0)
dftr98['Income_s_y'] = dftr98['Income_s_y'].fillna(0)
dftr98['Subsidy'] = dftr98['Subsidy'].fillna(0)
dftr98['sum03_incomes'] = dftr98['sum03_incomes'].fillna(0)
dftr98['total_income'] = dftr98['sum_netincome_w_y'] + dftr98['Income_s_y'] + dftr98['sum03_incomes'] + dftr98['Subsidy']
dftr98['total_income_month'] = dftr98['total_income'] / 12

## some of "cost" columns are object we need a function to convert them to int

In [ ]:
def object_to_int(x):
        try:
            return float(x)
        except:
            return 0

## last Function

In [16]:
def read_data(a,b):
    
    def custom_fasl(row):
        fasl_str = 'Fasl'
        for i in range(1,5):
            if ~np.isnan(row[fasl_str + str(i)]):
                row['Fasl1'] = row[fasl_str + str(i)]
                break
        return row
    
    def object_to_int(x):
        try:
            return float(x)
        except:
            return 0
    
    tr98=[]
    for file in pd.ExcelFile('data_sets/project1/'+a+b+'.xlsx').sheet_names:
       tr98.append(pd.read_excel('data_sets/project1/'+a+b+'.xlsx',sheet_name=file))
    
    for i in range(3, 21):
        for column in tr98[i].columns:
            tr98[i][column] = tr98[i][column].apply(object_to_int)
    
    
    dftr98=tr98[0]
    #dftr98=dftr98.merge(tr98[1],on='Address',how='left')
    dftr98=dftr98.merge(tr98[2],on='Address',how='left')
    for i in range(3,17):
        if i != 12:
            tr98[i]['value'] = tr98[i]['value'].apply(object_to_int)
            dftr98=dftr98.merge(tr98[i].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Cost'+str(i-2)}),on='Address',how='left')

    #house  price 1
    dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==41)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'HousePrice1'}),on='Address',how='left')
    #house price 2
    dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==42)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'HousePrice2'}),on='Address',how='left')
    # fuel
    dftr98=dftr98.merge(tr98[6][tr98[6]['code'].apply(lambda x:x//1000==45)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Fuel'}),on='Address',how='left')
    #car fuel
    dftr98=dftr98.merge(tr98[9][tr98[9]['code'].apply(lambda x:x//100==722)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Car_Fuel'}),on='Address',how='left')
    #benzin
    dftr98=dftr98.merge(tr98[9][tr98[9]['code'].apply(lambda x:x==72211)].groupby('Address').agg({'value':'sum'}).rename(columns={'value':'Benzin'}),on='Address',how='left')

    column_to_drop = ['MahMorajeh', 'TakmilDescA','TakmilDescB', 'TakmilDescC','JaygozinDescA','JaygozinDescB','JaygozinDescC','BlkAbdJaygozin','RadifJaygozin','Jaygozin', 'IsTel', 'FVam']
    existing_column = [col for col in column_to_drop if col in dftr98.columns]
    dftr98 = dftr98.drop(existing_column, axis = 1)

    dftr98[['HousePrice1', 'HousePrice2', 'Fuel', 'Car_Fuel', 'Benzin']] = dftr98[['HousePrice1', 'HousePrice2', 'Fuel', 'Car_Fuel', 'Benzin']].fillna(0)
    cost_str = 'Cost'
    for i in range(1,15):
        if i != 10:
            dftr98[cost_str+str(i)] = dftr98[cost_str+str(i)].fillna(0)

    # there are 8 types of material = [] ans i consider nans as the 9th type
    dftr98['material'] = dftr98['material'].fillna('9thtype')

    dftr98['Costs_sum'] = 0
    cost_str = 'Cost'
    for i in range(1,13):
        if i != 10:
            dftr98['Costs_sum'] += dftr98[cost_str + str(i)]
    dftr98['Costs_sum'] += dftr98[cost_str + "13"] / 12
    dftr98['Costs_sum'] += dftr98[cost_str + "14"] / 12
    dftr98 = dftr98.drop('Fasl', axis=1)
    temp = tr98[17].groupby('Address').agg({'netincome_w_m':'sum', 'netincome_w_y':'sum'}).rename(columns={'netincome_w_m':'sum_netincome_w_m'
    , 'netincome_w_y': 'sum_netincome_w_y'})
    temp_2 = tr98[17].groupby('Address')[['Fasl']].first()
    temp = temp.merge(temp_2, on='Address', how='left')
    dftr98 = dftr98.merge(temp, on='Address', how='left')
    #some of values in Fasl column are Nan its Because these family didnt mentioned in incomes(dftr98[17])
    dftr98 = dftr98.rename({'Fasl':'Fasl1'}, axis=1)

    temp = tr98[18].groupby('Address').agg({'income_s_y':'sum'}).rename(columns={'income_s_y':'Income_s_y'})
    tr98[18] = tr98[18].rename({'Fasl':'Fasl2'}, axis=1)
    temp_2 = tr98[18].groupby('Address')[['Fasl2']].first()
    temp = temp.merge(temp_2, on='Address', how='left')
    dftr98 = dftr98.merge(temp, on='Address', how='left')

    tr98[19]['income_pension'] = tr98[19]['income_pension'].apply(object_to_int)
    tr98[19]['income_rent'] = tr98[19]['income_rent'].apply(object_to_int)
    tr98[19]['income_transfer'] = tr98[19]['income_transfer'].apply(object_to_int)
    tr98[19]['income_aid'] = tr98[19]['income_aid'].apply(object_to_int)
    tr98[19]['income_interest'] = tr98[19]['income_interest'].apply(object_to_int)
    tr98[19]['income_resale'] = tr98[19]['income_resale'].apply(object_to_int)
    temp = tr98[19].groupby('Address').agg({'income_transfer':'sum', 'income_pension':'sum',
                                            'income_rent':'sum','income_aid':'sum',
                                            'income_interest':'sum', 'income_resale':'sum'})
    temp['sum03_incomes'] = temp['income_transfer']+temp['income_pension']+temp['income_rent']+temp['income_aid']+temp['income_interest']+temp['income_resale']
    temp.drop(['income_transfer', 'income_pension','income_rent','income_aid','income_interest', 'income_resale'], axis=1, inplace=True)
    tr98[19] = tr98[19].rename({'Fasl':'Fasl3'}, axis=1)
    temp_2 = tr98[19].groupby('Address')[['Fasl3']].first()
    temp = temp.merge(temp_2, on='Address', how='left')
    dftr98 = dftr98.merge(temp, on='Address', how='left')

    temp = tr98[20].groupby('Address').agg({'subsidy':'sum'}).rename(columns={'subsidy':'Subsidy'})
    tr98[20] = tr98[20].rename({'Fasl':'Fasl4'}, axis=1)
    temp_2 = tr98[20].groupby('Address')[['Fasl4']].first()
    temp = temp.merge(temp_2, on='Address', how='left')
    dftr98 = dftr98.merge(temp, on='Address', how='left')

    dftr98 = dftr98.apply(custom_fasl, axis=1)
    dftr98 = dftr98.drop(['Fasl2', 'Fasl3', 'Fasl4'], axis=1)
    #droping rows that doesnt have any income information
    dftr98 = dftr98.dropna(subset=['Fasl1'])

    dftr98['sum_netincome_w_m'] = dftr98['sum_netincome_w_m'].fillna(0)
    dftr98['sum_netincome_w_y'] = dftr98['sum_netincome_w_y'].fillna(0)
    dftr98['Income_s_y'] = dftr98['Income_s_y'].fillna(0)
    dftr98['Subsidy'] = dftr98['Subsidy'].fillna(0)
    dftr98['sum03_incomes'] = dftr98['sum03_incomes'].fillna(0)
    dftr98['total_income'] = dftr98['sum_netincome_w_y'] + dftr98['Income_s_y'] + dftr98['sum03_incomes'] + dftr98['Subsidy']
    dftr98['total_income_month'] = dftr98['total_income'] / 12
    dftr98.dropna(inplace=True)
    
    return dftr98

## reading all of the files and concating to eachother

In [17]:
df=[]
for a in ['R','U']:
    temp=[]
    for b in ['98','99','1400','1401']:
        print(a,b)
        x=read_data(a,b)
        x['year']=b
        temp.append(x)
    x=pd.concat(temp)
    x['state']=a
    df.append(x)
df=pd.concat(df)
df

R 98
R 99
R 1400
R 1401
U 98
U 99
U 1400
U 1401


,Address,weight,khanevartype,Takmil,province,town,tenure,room,space,construction,...,sum_netincome_w_m,sum_netincome_w_y,Fasl1,Income_s_y,sum03_incomes,Subsidy,total_income,total_income_month,year,state
0,20001383908,171.199524,1,1,Markazi,1,OwnedEstateLand,3,100.0,3.0,...,0.0,0.000000e+00,2.0,342000000.0,0.0,21840000.0,3.638400e+08,3.032000e+07,98,R
1,20001383911,171.199524,1,2,Markazi,1,OwnedEstateLand,3,70.0,3.0,...,0.0,0.000000e+00,2.0,0.0,43000000.0,10920000.0,5.392000e+07,4.493333e+06,98,R
2,20001383915,171.199524,1,2,Markazi,1,OwnedEstateLand,3,100.0,3.0,...,0.0,0.000000e+00,2.0,214500000.0,0.0,10920000.0,2.254200e+08,1.878500e+07,98,R
3,20001383916,171.199524,1,1,Markazi,1,OwnedEstateLand,4,100.0,3.0,...,18113334.0,2.363600e+08,2.0,0.0,0.0,21840000.0,2.582000e+08,2.151667e+07,98,R
4,20001383920,171.199524,1,2,Markazi,1,OwnedEstateLand,3,100.0,3.0,...,18000000.0,2.160000e+08,2.0,0.0,0.0,27300000.0,2.433000e+08,2.027500e+07,98,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19562,13006383827,2154.650617,1,1,Alborz,6,OwnedEstateLand,3,85.0,1.0,...,61966666.0,7.936000e+08,2.0,0.0,26220000.0,5040000.0,8.248600e+08,6.873833e+07,1401,U
19563,13006383832,2154.650617,1,1,Alborz,6,OwnedEstateLand,3,100.0,1.0,...,85800000.0,1.049600e+09,2.0,0.0,138370000.0,9135000.0,1.197105e+09,9.975875e+07,1401,U
19564,13006383835,2154.650617,1,1,Alborz,6,OwnedEstateLand,3,80.0,3.0,...,60000000.0,7.200000e+08,2.0,0.0,776400000.0,5040000.0,1.501440e+09,1.251200e+08,1401,U
19565,13006383838,2154.650617,1,2,Alborz,6,OwnedEstateLand,3,90.0,1.0,...,123933333.0,1.547200e+09,2.0,0.0,38370000.0,9135000.0,1.594705e+09,1.328921e+08,1401,U


## converting the df to csv file

In [ ]:
file_name = 'df.csv'
df.to_csv(file_name)